# Expand Context

The Chunk Nodes are individual chunks of text that used
to be part of an entire document.

You can reconstruct the original context by connecting the nodes
with relationships. It also makes the data easy to navigate and understand.

That is super helpful when you're building an application, for debugging and testing.

And, you can provide a better user experience. Your users will be able 
to directly interact with the data and even provide feedback that will
improve subsequent answers. 

You will create a connected context by making the following
changes to the knowledge graph:

1. Extract, create `(:Document)` nodes for each original source Form.
2. Enhance, add a summarized text property to each `(:Form)` node.
3. Expand, connect each `(:Chunk)` to the `(:Form)` node that it is part of

The graph will look like this...

```cypher
(:Document
  documentId: string
  documentUri: string
)

(:Section 
  sectionId: string //  a unique identifier for the form
  level: integer
  text: string
  summary: string // text summary generated with the LLM 
  summaryEmbedding: float[] // vector embedding of summary
  documentUri: string
  header1: string
  header2: string
  header3: string
)
```

```cypher
// Document contains sections
(:Document)-[:CONTAINS]->(:Section)

// Section contains Chunks
(:Section)-[:CONTAINS]->(:Chunk)
```

## Setup

In [3]:
%run 'shared.ipynb'

Connecting to Neo4j at bolt://neo4j-1:7687 as neo4j
Using data from /home/jovyan/data/single
Embedding with ollama using mxbai-embed-large
Chatting with ollama using llama3


# Extract Section Nodes

There will be one Document node for each document.

In [4]:
# Create parent sections from Chunk nodes

cypher_create_from_chunks = """
            MATCH (c:Chunk)
            WITH c, apoc.util.sha1([coalesce(c.header4, c.header3, c.header2, c.header1)]) AS sectionId
            MERGE (s:Section {sectionId: sectionId } )
            SET s.header1 = c.header1
            , s.header2 = c.header2
            , s.header3 = c.header3
            , s.documentUri = c.documentUri
            , s.level = CASE WHEN c.header4 IS NOT NULL THEN 4
                             WHEN c.header3 IS NOT NULL THEN 3
                             WHEN c.header2 IS NOT NULL THEN 2
                             WHEN c.header1 IS NOT NULL THEN 1
                        END 
            , s.text = coalesce(c.header4, c.header3, c.header2, c.header1)   
            MERGE (s)-[newRel:CONTAINS]->(c)
            """


In [5]:
gdb.execute_query(cypher_create_from_chunks)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffffa821cd50>, keys=[])

In [6]:
# Create all other sections based on the parent nodes created before

cypher_create_other_sections = """
            MATCH (e:Section)
            WHERE e.level = $level_to_create+1
            WITH e, CASE WHEN $level_to_create = 1 THEN e.header1
                    WHEN $level_to_create = 2 THEN e.header2
                    WHEN $level_to_create = 3 THEN e.header3
                    END as text
            WITH e, text, apoc.util.sha1([ text ]) AS sectionId    
            MERGE (s:Section {sectionId: sectionId} )
            set s.header1 = e.header1
                , s.header2 = e.header2
                , s.documentUri = e.documentUri
                , s.level = $level_to_create
                , s.text = text    
            MERGE (s)-[newRel:CONTAINS]->(e)
            """


In [7]:
gdb.execute_query(cypher_create_other_sections, level_to_create=3)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff55d65690>, keys=[])

In [8]:
gdb.execute_query(cypher_create_other_sections, level_to_create=2)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff55d66f90>, keys=[])

In [9]:
gdb.execute_query(cypher_create_other_sections, level_to_create=1)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff55d68650>, keys=[])

In [10]:
# Create a uniqueness constraint on the textId property of Text nodes 
gdb.execute_query('CREATE CONSTRAINT unique_section IF NOT EXISTS FOR (n:Section) REQUIRE n.sectionId IS UNIQUE')


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff55d69e10>, keys=[])

In [11]:
MATCH (c:Chunk)-[r:NEXT]->(c1:Chunk) DELETE r

SyntaxError: invalid syntax (1942152747.py, line 1)

## Extract Document nodes

You can now create a Document node for each of the distinct documentUri's of the level 1 nodes.


In [12]:
# Create all other sections based on the parent nodes created before

cypher_create_documents = """
            MATCH (s:Section)
            WHERE s.level = 1
            MERGE (d:Document {documentId: apoc.util.sha1([s.documentUri]) } )
            SET d.documentUri = s.documentUri
            MERGE (d)-[newRel:CONTAINS]->(s)
            """

gdb.execute_query(cypher_create_documents)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffffac144710>, keys=[])

# Parent retriever

In [20]:
search_results = neo4j_vector_search(
    'What is the size of the rink', 'chunks_vector'
)
for result in search_results:
    print (str(result["score"]) + " : " + result["node"]["chunkId"] + " : " + result["node"]["text"] )
    print ("------------")


Using vector index: chunks_vector
0.8374168276786804 : dd8ef53b27cfca8cc103b5d6f9631babc6368788 : The official size of the Rink shall be 60 m long and 26 m to 30 m wide. The corners shall be rounded in the arc of a circle with a radius of 7.0 m to 8.50 m. Any deviations from these dimensions for any IIHF competition require IIHF approval.
------------
0.8321495056152344 : 6a086313c89911cfff159d293c184b3c69346d16 : A “Penalty Shot” is designed to restore a scoring opportunity which was lost as a result of a foul being committed by the offending Team, based on the parameters set out in these rules.
------------
0.8314350247383118 : 87c26358c9d380d668a17604fd07dca4dde7bcc3 : The chest and arm protector worn by each Goalkeeper must be anatomically proportional and size specific based on the individual physical characteristics of that Goalkeeper.
------------
0.8215421438217163 : 539f2b88211090fbff8b3e2fdaa2be44d4d228d8 : The Rink shall be surrounded by a wall known as the “Boards”. The ide

In [15]:
retrieval_query_parent = """

    WITH node AS chunk, score AS score ORDER BY score 
    OPTIONAL MATCH (chunk:Chunk)<-[:CONTAINS]-(parent)
    OPTIONAL MATCH (parent)-[:CONTAINS]->(s:Chunk)
    WITH chunk, s, score ORDER BY s.chunkSeqId ASC
    WITH collect(s.text) as textList, chunk.text as text, score AS score
    RETURN apoc.text.join(textList, " \n ")  as text,
    score,  {} AS metadata 
    ORDER BY score desc

"""

vector_store_parent = Neo4jVector.from_existing_index(
    embedding=embeddings_api,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=retrieval_query_parent
)

# Create a retriever from the vector store
retriever_parent = vector_store_parent.as_retriever(search_kwargs={'k': 10})

# Create a chatbot Question & Answer chain from the retriever
chain_parent = prettifyChain(RetrievalQA.from_chain_type(
    chat_api, 
    chain_type="stuff", 
    retriever=retriever_parent,
    chain_type_kwargs={"verbose": True}
))


In [16]:
docs = retriever_parent.invoke("what happens if a player is injured")
docs

[Document(page_content='When a Player is injured or compelled to leave the ice during a Game, they may retire from the Game and be replaced by a substitute, but play must continue without the Teams leaving the ice.  \nDuring the play, if an injured Player wishes to retire from the ice and be replaced by a substitute, they must do so at the Players’ Bench and not through any other exit leading from the Rink. This is not a legal Player change and therefore when a violation occurs, a Bench-minor Penalty shall be imposed. \n If a penalized Player has been injured, they may proceed to the Dressing Room without taking a seat in the Penalty Box. The penalized Team shall immediately put a substitute Player in the Penalty Box, who shall serve the penalty until the injured Player is able to return to the game. They would replace their Teammate in the Penalty Box at the next stoppage of play.  \nFor violation of this rule, a Bench Minor Penalty shall be imposed. \n Should the injured penalized Pl

In [269]:
chain_parent("what happens if a player is injured")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
When a Player is injured or compelled to leave the ice during a Game, they may retire from the Game and be replaced by a substitute, but play must continue without the Teams leaving the ice.  
During the play, if an injured Player wishes to retire from the ice and be replaced by a substitute, they must do so at the Players’ Bench and not through any other exit leading from the Rink. This is not a legal Player change and therefore when a violation occurs, a Bench-minor Penalty shall be imposed. 
 If a penalized Player has been injured, they may proceed to the Dressing Room without taking a seat in the Penalty Box. The penalized Team shall immediately put a substitute Player in the Penalty Box, who sha

In [266]:


retrieval_query_parent = """
    WITH node AS chunk, score AS score ORDER BY score 
    OPTIONAL MATCH (chunk:Chunk)<-[:CONTAINS]-(parent)
    OPTIONAL MATCH (parent)-[:CONTAINS]->(s:Chunk)
    WITH chunk, s, score ORDER BY s.chunkSeqId ASC
    WITH collect(s.text) as textList, chunk.text as text, score AS score
    RETURN apoc.text.join(textList, " \n ")  as text,
    score,  {} AS metadata 
    ORDER BY score desc
"""


def neo4j_vector_search_2(question, retrieval_query):
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    CALL db.index.vector.queryNodes($index_name, $top_k, $question_embedding) 
        YIELD node, score
  """ + retrieval_query
  similar = []

  print ("Using vector index: " + str(VECTOR_INDEX_NAME))
    
  question_embedding = embeddings_api.embed_query(question)
  return gdb.execute_query(vector_search_query,
                      question=question, 
                      question_embedding=question_embedding, 
                      index_name=VECTOR_INDEX_NAME, 
                      top_k=10
                    ).records

search_results = neo4j_vector_search_2(
    'injured player', retrieval_query_parent
)
search_results[0]
    

Using vector index: chunks_vector


<Record text='When a Player is injured or compelled to leave the ice during a Game, they may retire from the Game and be replaced by a substitute, but play must continue without the Teams leaving the ice.  \nDuring the play, if an injured Player wishes to retire from the ice and be replaced by a substitute, they must do so at the Players’ Bench and not through any other exit leading from the Rink. This is not a legal Player change and therefore when a violation occurs, a Bench-minor Penalty shall be imposed. \n If a penalized Player has been injured, they may proceed to the Dressing Room without taking a seat in the Penalty Box. The penalized Team shall immediately put a substitute Player in the Penalty Box, who shall serve the penalty until the injured Player is able to return to the game. They would replace their Teammate in the Penalty Box at the next stoppage of play.  \nFor violation of this rule, a Bench Minor Penalty shall be imposed. \n Should the injured penalized Player who h

In [ ]:
MATCH (c:Chunk)<-[:CONTAINS]-(s:Section)
WHERE c.header4 = '8.1. INJURED PLAYER'
    RETURN c, s
    